## Indlejring

I vores tidligere eksempel arbejdede vi med højdimensionelle bag-of-words vektorer med længden `vocab_size`, og vi konverterede eksplicit lavdimensionelle positionsrepræsentationsvektorer til sparsomme one-hot repræsentationer. Denne one-hot repræsentation er ikke hukommelseseffektiv. Derudover behandles hvert ord uafhængigt af hinanden, så one-hot kodede vektorer udtrykker ikke semantiske ligheder mellem ord.

I denne enhed vil vi fortsætte med at udforske **News AG**-datasættet. For at starte, lad os indlæse dataene og få nogle definitioner fra den tidligere enhed.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Hvad er en embedding?

Ideen med **embedding** er at repræsentere ord ved hjælp af lavdimensionelle tætte vektorer, der afspejler ordets semantiske betydning. Vi vil senere diskutere, hvordan man bygger meningsfulde word embeddings, men for nu kan vi tænke på embeddings som en måde at reducere dimensionaliteten af en ordvektor.

En embedding-lag tager altså et ord som input og producerer en output-vektor med en specificeret `embedding_size`. På en måde minder det meget om et `Dense`-lag, men i stedet for at tage en one-hot-kodet vektor som input, kan det tage et ordnummer.

Ved at bruge et embedding-lag som det første lag i vores netværk kan vi skifte fra bag-of-words til en **embedding bag**-model, hvor vi først konverterer hvert ord i vores tekst til den tilsvarende embedding og derefter beregner en aggregeringsfunktion over alle disse embeddings, såsom `sum`, `average` eller `max`.

![Billede, der viser en embedding-klassifikator for fem sekvensord.](../../../../../translated_images/da/embedding-classifier-example.b77f021a7ee67eee.webp)

Vores klassifikator-neurale netværk består af følgende lag:

* `TextVectorization`-lag, som tager en streng som input og producerer en tensor af token-numre. Vi vil specificere en rimelig ordforrådsstørrelse `vocab_size` og ignorere mindre hyppigt anvendte ord. Inputformen vil være 1, og outputformen vil være $n$, da vi får $n$ tokens som resultat, hvor hver af dem indeholder numre fra 0 til `vocab_size`.
* `Embedding`-lag, som tager $n$ numre og reducerer hvert nummer til en tæt vektor af en given længde (100 i vores eksempel). Således vil input-tensoren med formen $n$ blive transformeret til en $n\times 100$ tensor.
* Aggregeringslag, som tager gennemsnittet af denne tensor langs den første akse, dvs. det vil beregne gennemsnittet af alle $n$ input-tensorer, der svarer til forskellige ord. For at implementere dette lag vil vi bruge et `Lambda`-lag og give det funktionen til at beregne gennemsnittet. Outputtet vil have formen 100 og vil være den numeriske repræsentation af hele inputsekvensen.
* Endeligt `Dense` lineært klassifikationslag.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


I `summary`-udskriften, i **output shape**-kolonnen, svarer den første tensor-dimension `None` til minibatch-størrelsen, og den anden svarer til længden af token-sekvensen. Alle token-sekvenser i minibatchen har forskellige længder. Vi vil diskutere, hvordan man håndterer dette i næste afsnit.

Lad os nu træne netværket:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Bemærk** at vi bygger en vektorisator baseret på et delmængde af dataene. Dette gøres for at fremskynde processen, og det kan resultere i en situation, hvor ikke alle tokens fra vores tekst er til stede i ordforrådet. I dette tilfælde vil disse tokens blive ignoreret, hvilket kan føre til en lidt lavere nøjagtighed. Dog giver en delmængde af tekst ofte en god estimering af ordforrådet i virkeligheden.


### Håndtering af variabel sekvensstørrelse

Lad os forstå, hvordan træning foregår i minibatches. I eksemplet ovenfor har input-tensoren dimension 1, og vi bruger minibatches med en længde på 128, så den faktiske størrelse af tensoren er $128 \times 1$. Antallet af tokens i hver sætning er dog forskelligt. Hvis vi anvender `TextVectorization`-laget på et enkelt input, er antallet af tokens, der returneres, forskelligt, afhængigt af hvordan teksten er tokeniseret:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Men når vi anvender vektoriseringen på flere sekvenser, skal den producere en tensor med rektangulær form, så den udfylder ubrugte elementer med PAD-tokenet (som i vores tilfælde er nul):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Her kan vi se indlejringerne:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Bemærk**: For at minimere mængden af udfyldning kan det i nogle tilfælde give mening at sortere alle sekvenser i datasættet i rækkefølge efter stigende længde (eller mere præcist, antal tokens). Dette vil sikre, at hver minibatch indeholder sekvenser af lignende længde.


## Semantiske indlejringer: Word2Vec

I vores tidligere eksempel lærte indlejringslaget at kortlægge ord til vektorrepræsentationer, men disse repræsentationer havde ikke nogen semantisk betydning. Det ville være rart at lære en vektorrepræsentation, hvor lignende ord eller synonymer svarer til vektorer, der ligger tæt på hinanden i forhold til en eller anden vektordistance (for eksempel euklidisk distance).

For at opnå dette skal vi fortræne vores indlejringsmodel på en stor samling tekst ved hjælp af en teknik som [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Den er baseret på to hovedarkitekturer, der bruges til at producere en distribueret repræsentation af ord:

 - **Continuous bag-of-words** (CBoW), hvor vi træner modellen til at forudsige et ord ud fra den omgivende kontekst. Givet ngrammet $(W_{-2},W_{-1},W_0,W_1,W_2)$ er målet for modellen at forudsige $W_0$ ud fra $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** er det modsatte af CBoW. Modellen bruger det omgivende vindue af kontekstord til at forudsige det aktuelle ord.

CBoW er hurtigere, og selvom skip-gram er langsommere, er det bedre til at repræsentere sjældne ord.

![Billede, der viser både CBoW- og Skip-Gram-algoritmer til at konvertere ord til vektorer.](../../../../../translated_images/da/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

For at eksperimentere med Word2Vec-indlejringen, der er fortrænet på Google News-datasættet, kan vi bruge **gensim**-biblioteket. Nedenfor finder vi de ord, der minder mest om 'neural'.

> **Note:** Når du først opretter ordvektorer, kan det tage noget tid at downloade dem!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Vi kan også udtrække vektorindlejringen fra ordet, som skal bruges til at træne klassifikationsmodellen. Indlejringen har 300 komponenter, men her viser vi kun de første 20 komponenter af vektoren for klarhed:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Det fantastiske ved semantiske indlejringer er, at du kan manipulere vektorindkodningen baseret på semantik. For eksempel kan vi bede om at finde et ord, hvis vektorrepræsentation er så tæt som muligt på ordene *konge* og *kvinde*, og så langt som muligt fra ordet *mand*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Et eksempel ovenfor bruger noget intern GenSym-magi, men den underliggende logik er faktisk ret simpel. En interessant ting ved indlejringer er, at du kan udføre normale vektoroperationer på indlejringsvektorer, og det ville afspejle operationer på ords **betydninger**. Eksemplet ovenfor kan udtrykkes i form af vektoroperationer: vi beregner vektoren svarende til **KONGE-MAND+KVINDE** (operationerne `+` og `-` udføres på vektorrepræsentationer af de tilsvarende ord), og derefter finder vi det nærmeste ord i ordbogen til den vektor:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Vi var nødt til at tilføje små koefficienter til *man*- og *woman*-vektorerne - prøv at fjerne dem for at se, hvad der sker.

For at finde den nærmeste vektor bruger vi TensorFlow-mekanik til at beregne en vektor af afstande mellem vores vektor og alle vektorer i ordforrådet, og derefter finde indekset for det minimale ord ved hjælp af `argmin`.


Mens Word2Vec virker som en fantastisk måde at udtrykke ords semantik på, har det mange ulemper, herunder følgende:

* Både CBoW- og skip-gram-modeller er **forudsigende indlejringer**, og de tager kun lokal kontekst i betragtning. Word2Vec udnytter ikke global kontekst.
* Word2Vec tager ikke højde for ords **morfologi**, dvs. det faktum, at ordets betydning kan afhænge af forskellige dele af ordet, såsom roden.

**FastText** forsøger at overvinde den anden begrænsning og bygger videre på Word2Vec ved at lære vektorrepræsentationer for hvert ord og de karakter-n-grammer, der findes inden for hvert ord. Værdierne af repræsentationerne gennemsnitliggøres derefter til én vektor ved hver træningsfase. Selvom dette tilføjer en masse ekstra beregning til prætræningen, gør det det muligt for ordindlejringer at kode sub-ord-information.

En anden metode, **GloVe**, bruger en anderledes tilgang til ordindlejringer, baseret på faktorisering af ord-kontekst-matricen. Først opbygges en stor matrix, der tæller antallet af ordforekomster i forskellige kontekster, og derefter forsøger den at repræsentere denne matrix i lavere dimensioner på en måde, der minimerer rekonstruktionsfejl.

Gensim-biblioteket understøtter disse ordindlejringer, og du kan eksperimentere med dem ved at ændre modelindlæsningskoden ovenfor.


## Brug af forudtrænede embeddings i Keras

Vi kan ændre eksemplet ovenfor for at forudfylde matricen i vores embedding-lag med semantiske embeddings, såsom Word2Vec. Ordforrådene fra den forudtrænede embedding og tekstkorpuset vil sandsynligvis ikke matche, så vi skal vælge ét. Her undersøger vi de to mulige muligheder: at bruge tokenizer-ordforrådet og at bruge ordforrådet fra Word2Vec-embeddings.

### Brug af tokenizer-ordforråd

Når vi bruger tokenizer-ordforrådet, vil nogle af ordene fra ordforrådet have tilsvarende Word2Vec-embeddings, og nogle vil mangle. Givet at vores ordforrådsstørrelse er `vocab_size`, og længden af Word2Vec embedding-vektoren er `embed_size`, vil embedding-laget blive repræsenteret af en vægtmatrix med formen `vocab_size`$\times$`embed_size`. Vi vil udfylde denne matrix ved at gennemgå ordforrådet:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


For ord, der ikke findes i Word2Vec-ordforrådet, kan vi enten lade dem være nul, eller generere en tilfældig vektor.

Nu kan vi definere et embedding-lag med fortrænede vægte:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Bemærk**: Bemærk, at vi sætter `trainable=False`, når vi opretter `Embedding`, hvilket betyder, at vi ikke genoplærer Embedding-laget. Dette kan medføre, at nøjagtigheden bliver en smule lavere, men det gør træningen hurtigere.

### Brug af embedding-ordforråd

Et problem med den tidligere tilgang er, at de ordforslag, der bruges i TextVectorization og Embedding, er forskellige. For at løse dette problem kan vi bruge en af følgende løsninger:
* Genoplære Word2Vec-modellen på vores ordforslag.
* Indlæse vores datasæt med ordforslaget fra den fortrænede Word2Vec-model. Ordforslag, der bruges til at indlæse datasættet, kan specificeres under indlæsningen.

Den sidstnævnte tilgang virker nemmere, så lad os implementere den. Først og fremmest vil vi oprette et `TextVectorization`-lag med det specificerede ordforslag, taget fra Word2Vec-embeddings:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Biblioteket gensim word embeddings indeholder en praktisk funktion, `get_keras_embeddings`, som automatisk vil oprette det tilsvarende Keras embeddings-lag for dig.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


En af grundene til, at vi ikke ser højere nøjagtighed, er fordi nogle ord fra vores datasæt mangler i den fortrænede GloVe-ordforråd, og derfor bliver de i bund og grund ignoreret. For at overvinde dette kan vi træne vores egne indlejringer baseret på vores datasæt.


## Kontekstuelle indlejringer

En vigtig begrænsning ved traditionelle forudtrænede indlejringsrepræsentationer som Word2Vec er, at selvom de kan fange en vis betydning af et ord, kan de ikke skelne mellem forskellige betydninger. Dette kan skabe problemer i efterfølgende modeller.

For eksempel har ordet 'play' forskellige betydninger i disse to sætninger:
- Jeg var til en **forestilling** på teatret.
- John vil gerne **lege** med sine venner.

De forudtrænede indlejringer, vi har talt om, repræsenterer begge betydninger af ordet 'play' i den samme indlejring. For at overvinde denne begrænsning skal vi bygge indlejringer baseret på **sproglige modeller**, som er trænet på en stor tekstsamling og *forstår*, hvordan ord kan sættes sammen i forskellige kontekster. At diskutere kontekstuelle indlejringer ligger uden for rammerne af denne tutorial, men vi vender tilbage til dem, når vi taler om sproglige modeller i næste enhed.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
